In [10]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional, TimeDistributed, InputLayer
from tensorflow.keras.layers import Embedding, Conv1D, Input, concatenate, SpatialDropout1D, Flatten, Conv2D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow import keras

import re
import tensorflow as tf
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter

In [11]:
# !wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

In [12]:
# !wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

In [13]:
neg_filepath = "negative.csv"
pos_filepath = "positive.csv"

neg_df = pd.read_csv(neg_filepath, sep=";", header=None)
pos_df = pd.read_csv(pos_filepath, sep=";", header=None)
df = pd.concat([neg_df, pos_df])
df = df[[3, 4]]

In [14]:
df.head()

,3,4
0,на работе был полный пиддес :| и так каждое за...,-1
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1
2,@elina_4post как говорят обещаного три года жд...,-1
3,"Желаю хорошего полёта и удачной посадки,я буду...",-1
4,"Обновил за каким-то лешим surf, теперь не рабо...",-1


In [15]:
df[4] = df[4].apply(lambda x: 0 if x == -1 else x)

In [16]:
import string

def preprocess(text):
    text = str(text).lower()
    text = re.sub('[^а-я]', ' ', text)
    pred = -1
    while pred != len(text):
        pred = len(text)
        text = text.replace("  ", " ")
    return text.lower()

def text_to_seq(text):
    return [i for i in text.split(" ") if not i.startswith("@") and 0 < len(i) < 20]


docs = []
for i in list(df[3].values):
    docs.append(text_to_seq(preprocess(i)))

df['norm'] = docs
print(df['norm'][:10])

0    [на, работе, был, полный, пиддес, и, так, кажд...
1    [коллеги, сидят, рубятся, в, а, я, из, за, дол...
2           [как, говорят, обещаного, три, года, ждут]
3    [желаю, хорошего, пол, та, и, удачной, посадки...
4    [обновил, за, каким, то, лешим, теперь, не, ра...
5    [кот, нка, вчера, носик, разбила, плакала, и, ...
6    [зашли, а, то, он, опять, затихарился, я, прям...
7      [а, вообще, я, не, болею, я, не, выздоравливаю]
8    [я, микрофраза, учимся, срать, кирпичами, в, р...
9    [я, хочу, с, тобой, помириться, но, сука, я, г...
Name: norm, dtype: object


In [17]:
d = {}

for seq in df['norm'].values:
    for word in seq:
        if word in d:
            d[word] += 1
        else:
            d[word] = 1
buf_d = {}
for k, v in d.items():
    if v > 4:
        buf_d[k] = v


# word2id = {word:i for i, word in enumerate(set([i for j in df['norm'].values for i in j if i in buf_d.keys()]))}
word2id = {word:i + 2 for i, word in enumerate(set([i for j in df['norm'].values for i in j if i in buf_d.keys()]))}
word2id['pad'] = 0
word2id['unk'] = 1  

id2word = {i:word for word, i in word2id.items()}

In [18]:
len(word2id)

31961

In [19]:
def data2ints(data, smth2id):
    int_data = []
    for seq in data:
        int_seq = []
        for i in seq:
            if i in smth2id.keys():
                int_seq.append(smth2id[i])

        int_data.append(int_seq)
    return int_data

In [20]:
sent_train, sent_test, y_train_ids, y_test_ids = train_test_split(df['norm'].values,
                                                              [[i] for i in df[4].values],
                                             
                                                                  test_size=0.2,
                                                              random_state=1)

In [21]:
# X_train_ids, X_test_ids, y_train_ids, y_test_ids = train_test_split(data2ints(df['norm'].values, word2id),
#                                                               [[i] for i in df[4].values],
#                                                               test_size=0.2, random_state=1)

X_train_ids, X_test_ids = data2ints(sent_train, word2id), data2ints(sent_test, word2id)
# y_train_ids, y_test_ids = data2ints(tag_train, 1), data2ints(tag_test, 1)

In [22]:
MAX_WORDS = max([len(i) for i in X_train_ids + X_test_ids])

X_train, X_test = pad_sequences(X_train_ids, maxlen=MAX_WORDS, padding='post'), pad_sequences(X_test_ids, maxlen=MAX_WORDS, padding='post')
y_train, y_test = pad_sequences(y_train_ids, 1), pad_sequences(y_test_ids, 1)
# y_train, y_test = pad_sequences(y_train_ids, maxlяen=sent_max_len, padding='post'), pad_sequences(y_test_ids, maxlen=sent_max_len, padding='post')

In [23]:
# y_train = to_categorical(y_train, num_classes=2)
# y_test = to_categorical(y_test, num_classes=2)
# y_train.shape


In [24]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


(181467, 32) (181467, 1) (45367, 32) (45367, 1)


In [25]:
X_train[5]



array([27372, 23365, 28588, 25518,  9854, 23559, 19272, 19510, 19145,
       15076, 31314, 13040,  8639,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)

In [35]:
metrics = [tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.Accuracy()]

In [44]:
word_in = Input(shape=(MAX_WORDS))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, mask_zero=True)(word_in)
lstm = Bidirectional(LSTM(64, recurrent_dropout=0.2))(emb_word)
out = Dense(1, activation="sigmoid")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 32)]              0         
_________________________________________________________________
embedding_9 (Embedding)      (None, 32, 100)           3196100   
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 128)               84480     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 3,280,709
Trainable params: 3,280,709
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epochs=3, verbose=1)

Train on 181467 samples, validate on 45367 samples
Epoch 1/3
181467/181467 [==============================] - 80s 441us/sample - loss: 0.5514 - precision: 0.7181 - recall: 0.7198 - accuracy: 0.0000e+00 - val_loss: 0.5151 - val_precision: 0.7379 - val_recall: 0.7585 - val_accuracy: 0.0000e+00
Epoch 2/3
181467/181467 [==============================] - 76s 421us/sample - loss: 0.4621 - precision: 0.7793 - recall: 0.7862 - accuracy: 0.0000e+00 - val_loss: 0.5197 - val_precision: 0.7302 - val_recall: 0.7709 - val_accuracy: 0.0000e+00
Epoch 3/3
181467/181467 [==============================] - 76s 420us/sample - loss: 0.4098 - precision: 0.8147 - recall: 0.8098 - accuracy: 0.0000e+00 - val_loss: 0.5385 - val_precision: 0.7444 - val_recall: 0.7355 - val_accuracy: 0.0000e+00


# load fasttext vectors

In [20]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model_fasttext = KeyedVectors.load(model_file)

In [21]:
weights = np.ndarray((len(word2id.keys()), 300))

for word in word2id.keys():
    weights[word2id[word]] = model_fasttext[word]
# [word2id[word]: model[word] for word in }

In [22]:
word_in = Input(shape=(MAX_WORDS))
emb_word = Embedding(input_dim=len(word2id), output_dim=300, trainable=False, weights=[weights])(word_in)
lstm = Bidirectional(LSTM(64, recurrent_dropout=0.2))(emb_word)
out = Dense(1, activation="sigmoid")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 32, 300)           9588300   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               186880    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 9,775,309
Trainable params: 187,009
Non-trainable params: 9,588,300
_________________________________________________________________


In [23]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epoch s=3, verbose=1)

Train on 181467 samples, validate on 45367 samples
Epoch 1/3
181467/181467 [==============================] - 62s 340us/sample - loss: 0.6415 - precision: 0.6262 - recall: 0.6647 - accuracy: 0.0000e+00 - val_loss: 0.6152 - val_precision: 0.6490 - val_recall: 0.7119 - val_accuracy: 0.0000e+00
Epoch 2/3
181467/181467 [==============================] - 60s 330us/sample - loss: 0.6081 - precision: 0.6657 - recall: 0.6909 - accuracy: 0.0000e+00 - val_loss: 0.6055 - val_precision: 0.6435 - val_recall: 0.7549 - val_accuracy: 0.0000e+00
Epoch 3/3
181467/181467 [==============================] - 61s 338us/sample - loss: 0.5990 - precision: 0.6722 - recall: 0.6965 - accuracy: 0.0000e+00 - val_loss: 0.5981 - val_precision: 0.6498 - val_recall: 0.7557 - val_accuracy: 0.0000e+00


# Train fasttext model

In [24]:
from gensim.models import KeyedVectors, FastText

ft_model = FastText(size=100)
ft_model.build_vocab(sentences=df['norm'].values)
ft_model.train(sentences=df['norm'].values, total_examples=len(df['norm'].values), epochs=5)

In [25]:
weights_train = np.array([ft_model.wv[word] for word in id2word.values()])


In [26]:
word_in = Input(shape=(MAX_WORDS))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, trainable=False, weights=[weights_train])(word_in)
lstm = Bidirectional(LSTM(64, recurrent_dropout=0.2))(emb_word)
out = Dense(1, activation="sigmoid")(lstm)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 32, 100)           3196100   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 3,280,709
Trainable params: 84,609
Non-trainable params: 3,196,100
_________________________________________________________________


In [27]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epochs=3, verbose=1)

Train on 181467 samples, validate on 45367 samples
Epoch 1/3
181467/181467 [==============================] - 65s 361us/sample - loss: 0.6743 - precision: 0.5757 - recall: 0.6360 - accuracy: 0.0000e+00 - val_loss: 0.6582 - val_precision: 0.5935 - val_recall: 0.6919 - val_accuracy: 0.0000e+00
Epoch 2/3
181467/181467 [==============================] - 60s 333us/sample - loss: 0.6521 - precision: 0.6116 - recall: 0.6624 - accuracy: 0.0000e+00 - val_loss: 0.6421 - val_precision: 0.6050 - val_recall: 0.7277 - val_accuracy: 0.0000e+00
Epoch 3/3
181467/181467 [==============================] - 61s 335us/sample - loss: 0.6386 - precision: 0.6291 - recall: 0.6646 - accuracy: 0.0000e+00 - val_loss: 0.6386 - val_precision: 0.6021 - val_recall: 0.7815 - val_accuracy: 0.0000e+00


## Concat model (learning embedding layer + char embedding layer with CNN)

In [26]:
filtered_vocab = set([word for sent in df['norm'].values for word in sent])

chars = set([letter for word in filtered_vocab for letter in word])
n_chars = len(chars)
print(list(chars)[:30])

['д', 'щ', 'с', 'а', 'р', 'й', 'э', 'ю', 'ф', 'г', 'б', 'ш', 'у', 'е', 'к', 'н', 'ц', 'ы', 'о', 'л', 'ч', 'ь', 'з', 'в', 'и', 'х', 'м', 'ж', 'ъ', 'я']


In [27]:
char2id = {c: i + 2 for i, c in enumerate(chars)}
char2id["pad"] = 0
char2id["unk"] = 1

id2char = {i:char for char, i in char2id.items()}

In [28]:
char_max_len = max(len(x) for x in filtered_vocab)
print("максимальная длина слова:", char_max_len)

максимальная длина слова: 19


In [29]:
def data2ints(data, smth2id):
    int_data = []
    for seq in data:
        int_seq = []
        for i in seq:
            int_seq.append(smth2id.get(i.lower(), 1))

        int_data.append(int_seq)
    return int_data


In [30]:
# X_train_ids, X_test_ids = data2ints(sent_train, word2id), data2ints(sent_test, word2id)
# y_train_ids, y_test_ids = data2ints(tag_train, tag2id), data2ints(tag_test, tag2id)


print(X_train_ids[0])
print(y_train_ids[0])

print(X_test_ids[0])
print(y_test_ids[0])

[14285, 13787, 4897, 7603, 29415, 11957, 22228]
[0]
[29415, 13040, 14363, 1537, 14363, 22472, 2604, 31388, 1392]
[1]


In [31]:
sent_max_len = MAX_WORDS

def make_X_char(sentences):
    X_char = []
    
    for sentence in sentences:
        sent_seq = []
        for i in range(sent_max_len):
            word_seq = []
            for j in range(char_max_len):
                try:
                    word_seq.append(char2id[sentence[i][j].lower()])
                except:
                    word_seq.append(char2id["pad"])
            sent_seq.append(word_seq)
        X_char.append(np.array(sent_seq))
    return np.array(X_char)

In [32]:
X_char_train, X_char_test = make_X_char(sent_train), make_X_char(sent_test)


In [33]:
X_char_train.shape

(181467, 32, 19)

In [38]:
# один вход для слов
word_in = Input(shape=(sent_max_len))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, mask_zero=True)(word_in)
word_lstm = Bidirectional(LSTM(units=64, recurrent_dropout=0.2))(emb_word)

# другой вход для символов
char_in = Input(shape=(sent_max_len, char_max_len))
emb_char = TimeDistributed(Embedding(input_dim=len(char2id), output_dim=10, input_length=char_max_len))(char_in)
# свертка на символах (CharRNN) применяется к каждому слову отдельно
char_enc = TimeDistributed(Conv1D(filters=12, kernel_size=3))(emb_char)
char_flat = TimeDistributed(Flatten())(char_enc)
char_lstm = Bidirectional(LSTM(units=64, recurrent_dropout=0.2))(char_flat)

# LSTM проходится по всей последовательности, на каждом шаге беря эмбеддинг слова по словарю + символьный эмбеддинг слова 
x = concatenate([word_lstm, char_lstm])
out = Dense(1, activation="sigmoid")(x)

model = Model(inputs=[char_in, word_in], outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 19)]     0                                            
__________________________________________________________________________________________________
time_distributed_6 (TimeDistrib (None, 32, 19, 10)   340         input_6[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
time_distributed_7 (TimeDistrib (None, 32, 17, 12)   372         time_distributed_6[0][0]         
____________________________________________________________________________________________

In [39]:
model.fit([X_char_train, X_train],
          y_train,
          validation_data=([X_char_test, X_test], y_test), batch_size=512, epochs=3, verbose=1)

Epoch 1/3
355/355 [==============================] - 109s 277ms/step - loss: 0.5966 - precision: 0.6670 - recall: 0.6984 - accuracy: 0.0000e+00 - val_loss: 0.5141 - val_precision: 0.7413 - val_recall: 0.7601 - val_accuracy: 0.0000e+00
Epoch 2/3
355/355 [==============================] - 100s 282ms/step - loss: 0.4541 - precision: 0.7885 - recall: 0.7927 - accuracy: 0.0000e+00 - val_loss: 0.5129 - val_precision: 0.7442 - val_recall: 0.7590 - val_accuracy: 0.0000e+00
Epoch 3/3
355/355 [==============================] - 101s 284ms/step - loss: 0.3989 - precision: 0.8229 - recall: 0.8187 - accuracy: 0.0000e+00 - val_loss: 0.5344 - val_precision: 0.7535 - val_recall: 0.7203 - val_accuracy: 0.0000e+00


## Подготовка датасета, формирование матриц признаков -- 1 балл

Проведена предобработка текстов и сформированы матрицы признаков

## Внятное сравнение моделей между собой по качеству -- 1 балл

Наилучший результат показывает модель из первого пункта (обучающийся слой векторизации), который показал результат:

**val_precision: 0.7444 - val_recall: 0.7355**

Остальные модели (предзагруженный fasttext **val_precision: 0.6498 - val_recall: 0.7557** и обученный fasttext **val_precision: 0.6021 - val_recall: 0.7815**) показали результат хуже по **precision**, но зато лучше по **recall**.

Сконкатенированная модель (обучающийся слой векторизации по словам + посимвольный вход) показала самый высокий результат среди всех моделей **val_precision: 0.7442 - val_recall: 0.7590**, хоть и переобучилась уже на 3-ей эпохе

## Показано, что качество можно улучшить засчет работы с гиперпараметрами и/или доведения архитектуры (помогло изменение размера слоя/эмбеддингов, добавили Dropout и спаслись от переобучения, добавили слоев и т.д.) -- 1 балл

Улучшим первую модель (обучающийся слой векторизации) путём добавления второго lstm слоя и увеличения юнитов в них:

In [42]:
word_in = Input(shape=(MAX_WORDS))
emb_word = Embedding(input_dim=len(word2id), output_dim=100, mask_zero=True)(word_in)
lstm_1 = Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.2))(emb_word)
lstm_2 = Bidirectional(LSTM(128, recurrent_dropout=0.2))(lstm_1)
out = Dense(1, activation="sigmoid")(lstm_2)

model = Model(inputs=word_in, outputs=out)

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=metrics)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 32, 100)           3196100   
_________________________________________________________________
bidirectional_9 (Bidirection (None, 32, 256)           234496    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 256)               394240    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 3,825,093
Trainable params: 3,825,093
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=512, epochs=3, verbose=1)

Epoch 1/3
355/355 [==============================] - 241s 644ms/step - loss: 0.5856 - precision: 0.7080 - recall: 0.7311 - accuracy: 0.0000e+00 - val_loss: 0.5151 - val_precision: 0.7303 - val_recall: 0.7759 - val_accuracy: 0.0000e+00
Epoch 2/3
355/355 [==============================] - 237s 668ms/step - loss: 0.4450 - precision: 0.7912 - recall: 0.7952 - accuracy: 0.0000e+00 - val_loss: 0.5238 - val_precision: 0.7451 - val_recall: 0.7419 - val_accuracy: 0.0000e+00
Epoch 3/3
355/355 [==============================] - 238s 670ms/step - loss: 0.3752 - precision: 0.8346 - recall: 0.8330 - accuracy: 0.0000e+00 - val_loss: 0.5504 - val_precision: 0.7384 - val_recall: 0.7443 - val_accuracy: 0.0000e+00


На второй эпохе обучения, скор на валидации составил **val_precision: 0.7451 - val_recall: 0.7419**, что выше, чем у базовой модели **val_precision: 0.7444 - val_recall: 0.7355**